In [3]:
pip install langid

In [4]:
import pandas as pd
import gzip
import json
import nltk
import string
from collections import Counter
import langid
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1) Можно использовать bootstrapping в широком понимании. В нашем случае с отзывами это значило бы найти довольно большое количество шаблонов типа 'I just adore X', 'X is my favourite', 'I have used X for ...' и выделить из них то, что стоит на месте Х. Для этого способа нужно пройтись по корпусу (а он у нас очень большой!) и вручную выделить нужные шаблоны. Итак, данные: сам корпус + выделенные на его основании шаблоны.
<br>Проблем с эти способом, кажется, больше, чем его преимуществ: 
<br>- не все пользователи четко и ясно пишут, что им нравится этот продукт/это их любимый продукт. Формулировок может быть практически бесконечное множество, собрать все (или почти все) возможные шаблоны займет очень много времени.
<br>- проблемы с сущностями, состоящими из 2+ слов. Например, один пользователь напишет 'I like Aveda', другой 'I like Aveda smoother', третий 'I like Aveda Smooth Infusion Hair Freeze Smoother'. Кажется, что во всех этих случаях мы выделим только Aveda, потеряв остальные части сущности. Если пытаться выделить больше, то можно наоборот забрать лишние ненужные слова (трудно жить без морф. информации!)
<br>- найденные сущности смешаются с найденные дескрипторами и отделить друг от друга не вручную будет малореально
<br>+ процесс четко контролируемый, если мы увидим, что выдается что-то не то, можно будет отследить это и изменить/убрать соответствующий шаблон
<br><br>2) Можно самим составить словарь некоторых сущностей/дескрипторов, основываясь на метаданных (названиях продуктов) и расширить его с помощью эмбеддингов (для дескрипторов можно использовать как обученные на корпусе, так и предобученные, для сущностей -- только обученные на нашем корпусе). Нам понадобится: корпус + метаданные + выделенные на основании метаданных дескрипторы и некоторые сущности
<br>- скорее всего, мы потеряем редко встречающиеся сущности и сущности, использующиеся в необычных контекстах
<br>- здесь мы по сути используем черный ящик: если нам выдастся что-то совсем неподходящее, мы вряд ли сможем понять, почему это произошло и устранить причину, придется подчищать результаты от всего мусора вручную
<br>+ не нужно вручную искать и задавать все возможные контексты использования сущностей, как в первом варианте; будут учтены не только "захардкоженные" нами контексты, но и контексты, немного отличающиеся друг от друга 
<br>+ можно отдельно смотреть дескрипторы+синонимы(векторные) и сущности+синонимы(векторные), если нам это интересно (а это может быть полезно для работы с коллокациями, содержащими сущности)
<br><br>3) Этот способ похож одновременно на первый и второй, но также использует морфологическую информацию. Можно составить список возможных дескрипторов, основываясь на метаданных, затем составить с ними шаблоны, использующиеся морфологическую информацию. yargy тут подойдет плохо, тк он не работает так с английским, но мы будем использовать библиотеку spaCy, работающую примерно также. В качестве сущностей будем выбирать имена собственные. Тут нам понадобится корпус + метаданные + выделенные на основе метаданных дескрипторы.
<br>- список дескрипторов зависит только от нас и должен быть максимально полным: если мы упустим дескриптор, мы упустим все принадлежащие ему сущности
<br>- теряем сущности, которые ни разу не попались рядом с дескриптором, хотя другими методами могли бы их найти
<br>+ дескрипторы отдельно, именованные сущности отдельно
<br>+ можем контролировать и корректировать шаблоны, не черный ящик
<br>+ учитываем морф.информацию, будет меньше мусора
<br>+ можем брать сколько угодно большие сущности, хоть из 6 слов, главное, чтоб они шли рядом с дескриптором
<br>+ выделять надо только дескрипторы, а не дескрипторы и сущности, их не так много, тк у нашего корпуса четкая однородная тематика

Этой функцией мы будем предобрабатывать текст, отделяя знаки препинания, приводя все к нижнему регистру и избавляясь от стоп слов.

In [81]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", text)
    text = [x for x in text.split(' ') if x not in stopwords.words('english')]
    return ' '.join(text)

Скачиваем данные, я беру датасеты с люксовой косметикой.

In [6]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Luxury_Beauty.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz

--2021-12-04 18:18:41--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Luxury_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7608964 (7.3M) [application/octet-stream]
Saving to: ‘meta_Luxury_Beauty.json.gz.1’

meta_Luxury_Beauty. 100%[===================>]   7.26M  6.16MB/s    in 1.2s    

2021-12-04 18:18:43 (6.16 MB/s) - ‘meta_Luxury_Beauty.json.gz.1’ saved [7608964/7608964]

--2021-12-04 18:18:43--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Luxury_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73193215 (70M) [application/octet-stream]
Saving to: ‘Luxury_Beauty.json.gz.1’

Luxury_Beauty.json. 100%[===================>]  69.80M

In [7]:
data = []
with gzip.open('meta_Luxury_Beauty.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

data1 = []
with gzip.open('Luxury_Beauty.json.gz') as f:
    for l in f:
        data1.append(json.loads(l.strip()))

In [8]:
df = pd.DataFrame.from_dict(data1)
df_meta = pd.DataFrame.from_dict(data)

Я также скачала метаданные. К сожалению, я сделала первую часть дз до того, как узнала, что тут можно было извлекать сущности автоматически, поэтому я просто посмотрела на самые популярные биграммы и триграммы в заголовках, и на основании них сделала выводы о популярных дескрипторах. Закомменчу и скрою вывод с биграммами и триграммами, чтобы не засорять тетрадку.

In [9]:
titles = ' '.join(list(df_meta['title']))

In [10]:
tokens = [x for x in nltk.word_tokenize(titles) if x not in string.punctuation]
c = Counter(tokens).most_common()
bgs = nltk.bigrams(tokens)
fdist = nltk.FreqDist(bgs)
freq_bi = []
for k,v in fdist.items():
    freq_bi.append([' '.join(k), v])
freq_bi = sorted(freq_bi, key = lambda x: int(x[1]), reverse=True)
tgs = nltk.trigrams(tokens)
fdist = nltk.FreqDist(tgs)
freq_tri = []
for k,v in fdist.items():
    freq_tri.append([' '.join(k), v])
freq_tri = sorted(freq_tri, key = lambda x: int(x[1]), reverse=True)

In [11]:
#[x for x in freq_bi if 'fl' not in x[0] and 'oz' not in x[0] and 'Oz' not in x[0]]

In [12]:
#[x for x in freq_tri if 'fl' not in x[0] and 'oz' not in x[0] and 'Oz' not in x[0]]

Немного предобработаю тексты отзывов, избавлюсь от совсем коротких, они явно нам не пригодятся (тк там не будет либо названия продукта, либо полезной информации о нем, а скорее всего и того и того). Разделю корпус отзывов на чанки по 1000 штук, чтобы было легче и удобнее их обрабатывать. МСами отзывы в корпусе разделю \n.

In [108]:
df = df.dropna(subset=['reviewText'])
reviews = list([str(x) for x in df['reviewText']])
reviews = [x for x in reviews if len(list(x))>8]
chunks_pre = [reviews[x:x+100] for x in range(0, len(reviews), 1000)]
chunks = [' \n '.join(x) for x in chunks_pre]
chunks1 = [' /// '.join(x) for x in chunks_pre]

Список полученных мной возможных дескрипторов. В первом списке самостоятельные дескрипторы, во втором и третьем -- дескрипторы, которые обычно идут в паре друг в другом (соотетственно 1-й и 2-й).

In [14]:
descriptors = ['product',
    'perfume',
    'spray',
    'cream',
    'lotion',
    'gloss',
    'glosses',
    'sunscreen',
    'tan',
    'moisturizer',
    'moisturiser',
    'scrub',
    'mascara',
    'powder',
    'conditioner',
    'shampoo',
    'gel',
    'cleanser',
    'mask',
    'treatment',
    'brush',
    'brushes',
    'pencil',
    'toothpaste',
    'foundation',
    'wax',
    'waxes',
    'balm',
    'detangler',
    'clay',
    'bb'
]

first_descriptors = [
    'nail',
    'hand',
    'eau',
    'hair',
    'lip',
    'body',
    'face',
    'hair'
]

second_descriptors = [
    'polish',
    'polishes',
    'cream',
    'de',
    'dryer',
    'balm',
    'oil',
    'butter',
    'wash',
    'washes',
    'lotion'
]

all_descs = descriptors + [x + 's' for x in descriptors] + first_descriptors + second_descriptors + ['parfum', 'toilette']

Поскольку yargy нормально работает только с русским языком, я вопсользуюсь аналогичной функцией spaCy. Буду искать идущие рядом с дескрипторами имена собственным (PROPN), возможно в комбинации с прилагательными, существительными и тд. При этом не учитываю меры измерения fl и oz, таким образом отсекая мусор. В процессе еще проверяю язык отзыва: из-за этого мы потеряем некоторые короткие отзывы, зато отсеется довольно много мусора на испанском, который spaCy идентифицировал на PROPN и который соответственно засорял наш список сущностей.

In [15]:
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab, validate=True)
pattern = [[{"LEMMA": {"IN": descriptors}}, {"POS": "NOUN", "OP": "?"}, {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, 
            {"POS": {"IN": ['PUNCT', 'SYMB']}, "OP": "?"}, {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}],
           [{"POS": "NOUN", "OP": "?"}, {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, {"POS": {"IN": ['PUNCT', 'SYMB']}, "OP": "?"},
            {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, {"POS": "JJ", "OP": "*"}, {"LEMMA": {"IN": descriptors}}],
           [{"POS": "NOUN", "OP": "?"}, {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, {"POS": {"IN": ['PUNCT', 'SYMB']}, "OP": "?"},
            {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, {"POS": "JJ", "OP": "*"}, {"LEMMA": {"IN": first_descriptors}},
            {"LEMMA": {"IN": second_descriptors}}, {"LEMMA": {"IN": ['parfum', 'toilette']}, "OP": "?"}],
           [{"LEMMA": {"IN": first_descriptors}}, {"LEMMA": {"IN": second_descriptors}}, {"LEMMA": {"IN": ['parfum', 'toilette']}, "OP": "?"},
            {"POS": "NOUN", "OP": "?"}, {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}, {"POS": {"IN": ['PUNCT', 'SYMB']}, "OP": "?"},
            {"LEMMA": {"NOT_IN": ['fl', 'oz', 'FL', 'OZ']}, "POS": "PROPN", "OP": "+"}]]
matcher.add("beautyproduct", pattern)

nes = []
nes_desc = []
for chunk in tqdm(chunks):
  for sent in chunk.split('\n'):
    doc = nlp(sent)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  
        span = doc[start:end] 
        if langid.classify(sent)[0]=='en':
          nes_desc.append(span.text)
          no_desc = span.text.split()
          for i in all_descs:
            if i in no_desc:
              no_desc.remove(i)
          nes.append(' '.join(no_desc))

100%|██████████| 558/558 [14:58<00:00,  1.61s/it]


Я на всякий случай получаю два списка: ner, то есть сами найденные сущности без дескрипторов, и ner_desc, то есть найденные сущности + их дескрипторы.

In [16]:
nes = [x for x in nes if len(x) > 1]

Оставляю по 20 самых частотных сущностей. Кстати, распечатаем их, чтоб понять, адекватно ли у нас сработал поиск.

In [17]:
c = Counter()
nes_desc_common = Counter(nes_desc).most_common(20)
nes_common = Counter(nes).most_common(20)
nes_set = set(nes)
nes_desc_set = set(nes_desc)

In [18]:
nes_desc_common

[('Mario Badescu products', 38),
 ('Jane Iredale products', 36),
 ('Paul Mitchell products', 24),
 ('Jack Black products', 20),
 ('Badger Hair brush', 16),
 ('Escali Badger Hair brush', 16),
 ('Juice Beauty products', 15),
 ('Hot Tools products', 12),
 ('Mason Pearson brush', 12),
 ('Paul Mitchell product', 8),
 ('Jane Iredale product', 7),
 ('Jack Black product', 7),
 ('Roche-Posay products', 7),
 ('Mason Pearson brushes', 6),
 ('La Roche-Posay products', 6),
 ('Anthony Logistics products', 5),
 ('Jack Black shampoo', 5),
 ('Bare Minerals powders', 5),
 ('Iredale BB cream', 4),
 ('Jane Iredale BB cream', 4)]

In [19]:
nes_common

[('Jane Iredale', 47),
 ('Jack Black', 44),
 ('Mario Badescu', 44),
 ('Paul Mitchell', 41),
 ('Juice Beauty', 20),
 ('Mason Pearson', 18),
 ('Badger Hair', 16),
 ('Escali Badger Hair', 16),
 ('Hot Tools', 15),
 ('Bare Minerals', 12),
 ('Roche-Posay', 10),
 ('La Roche-Posay', 9),
 ('Billy Jealousy', 6),
 ('Anthony Logistics', 5),
 ('Iredale BB', 5),
 ('Jane Iredale BB', 5),
 ('Elizabeth Arden', 5),
 ('La Roche', 5),
 ('Molton Brown', 5),
 ('jane iredale', 4)]

Предобрабатываем полученные сущности и готовим их к поиску биграмм -- прилепляем друг к другу через _, чтоб они воспринимались программой как одно слово.

In [20]:
nes_desc_prep = [preprocess_text(x[0]) for x in tqdm(nes_desc_common)]
nes_prep = [preprocess_text(x[0]) for x in tqdm(nes_common)]
nes_split = ['_'.join(x.split()) for x in nes_prep]
nes_desc_split = ['_'.join(x.split()) for x in nes_desc_prep]

100%|██████████| 20/20 [00:00<00:00, 1159.91it/s]


Предобрабатываем тексты отзывов.

In [ ]:
chunks_prep1 = [preprocess_text(c) for c in tqdm(chunks1)]

Заменяем найденные сущности на такие же сущности, но через _ в тексте отзывов. Извлекаем биграммы, ранжируем с помощью трех коллокационных метрик (pmi, log likelyhood, dice). Сначала посмотрим, что получается для сущностей без дескрипторов, потом -- что для сущностей с дескрипторами.

In [129]:
chunks_new = []
for chunk in tqdm(chunks_prep1):
  for ne in nes_prep:
    ne_split = ne.split()
    if ne in chunk:
      chunk = chunk.replace(ne, '_'.join(ne_split))
  chunks_new.append(chunk)


100%|██████████| 558/558 [00:00<00:00, 3147.49it/s]


In [ ]:
list_metrics = {}

for ner in tqdm(nes_split):
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  creature_filter = lambda *w: ner not in w 
  finder = BigramCollocationFinder.from_words(' '.join(chunks_new).split())
  finder.apply_freq_filter(2)
  finder.apply_ngram_filter(creature_filter)
  list_metrics[ner] = (finder.nbest(bigram_measures.pmi, 15), finder.nbest(bigram_measures.likelihood_ratio, 15), finder.nbest(bigram_measures.dice, 15))

In [131]:
for i in list_metrics:
    print('word: ', i)
    print('_______')
    print('pmi results: ')
    for result in list_metrics[i][0][:5]:
      print(' '.join(result))
    print('_______')
    print('log likelihood results: ')
    for result in list_metrics[i][1][:5]:
      print(' '.join(result))
    print('_______')
    print('dice results: ')
    for result in list_metrics[i][2][:5]:
      print(' '.join(result))
    print('\n\n')

word:  jane_iredale
_______
pmi results: 
jane_iredale purepressed
jane_iredale pressed
sells jane_iredale
jane_iredale cosmetics
jane_iredale pure
_______
log likelihood results: 
jane_iredale products
love jane_iredale
jane_iredale pressed
jane_iredale purepressed
jane_iredale website
_______
dice results: 
jane_iredale pressed
jane_iredale purepressed
jane_iredale website
jane_iredale pure
jane_iredale products



word:  jack_black
_______
pmi results: 
jack_black industrial
jack_black beard
jack_black pomades
"> jack_black
jack_black clay
_______
log likelihood results: 
jack_black products
jack_black beard
jack_black true
"> jack_black
jack_black lip
_______
dice results: 
jack_black beard
"> jack_black
jack_black true
jack_black clay
jack_black industrial



word:  mario_badescu
_______
pmi results: 
mario_badescu enzyme
mario_badescu buffering
mario_badescu rose
mario_badescu vitamin
fan mario_badescu
_______
log likelihood results: 
mario_badescu products
mario_badescu bufferin

In [25]:
chunks_new = []
for chunk in tqdm(chunks_prep1):
  for ne in nes_desc_prep:
    ne_split = ne.split()
    if ne in chunk:
      chunk = chunk.replace(ne, '_'.join(ne_split))
  chunks_new.append(chunk)

100%|██████████| 558/558 [00:00<00:00, 2302.23it/s]


In [26]:
list_metrics_desc = {}

for ner in tqdm(nes_desc_split):
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  creature_filter = lambda *w: ner not in w 
  finder = BigramCollocationFinder.from_words(' '.join(chunks_new).split())
  finder.apply_freq_filter(2)
  finder.apply_ngram_filter(creature_filter)
  list_metrics_desc[ner] = (finder.nbest(bigram_measures.pmi, 15), finder.nbest(bigram_measures.likelihood_ratio, 15), finder.nbest(bigram_measures.dice, 15))

100%|██████████| 20/20 [02:34<00:00,  7.75s/it]


In [27]:
for i in list_metrics_desc:
    print('word: ', i)
    print('_______')
    print('pmi results: ')
    for result in list_metrics_desc[i][0][:5]:
      print(' '.join(result))
    print('_______')
    print('log likelihood results: ')
    for result in list_metrics_desc[i][1][:5]:
      print(' '.join(result))
    print('_______')
    print('dice results: ')
    for result in list_metrics_desc[i][2][:5]:
      print(' '.join(result))
    print('\n\n')

word:  mario_badescu_products
_______
pmi results: 
other mario_badescu_products
mario_badescu_products are
all mario_badescu_products
love mario_badescu_products
of mario_badescu_products
_______
log likelihood results: 
other mario_badescu_products
mario_badescu_products .
mario_badescu_products and
mario_badescu_products are
all mario_badescu_products
_______
dice results: 
other mario_badescu_products
mario_badescu_products are
all mario_badescu_products
love mario_badescu_products
of mario_badescu_products



word:  jane_iredale_products
_______
pmi results: 
jane_iredale_products are
love jane_iredale_products
using jane_iredale_products
all jane_iredale_products
like jane_iredale_products
_______
log likelihood results: 
love jane_iredale_products
jane_iredale_products are
using jane_iredale_products
jane_iredale_products and
all jane_iredale_products
_______
dice results: 
jane_iredale_products are
love jane_iredale_products
using jane_iredale_products
all jane_iredale_products

Столкнулись с очевидной проблемой. Когда смотрим на сущности без дескрипторов, в коллокации постоянно влезают дескрипторы, которые для отчета о пользовательском восприятии товара будет совсем неинформативны. Когда смотрим на сущности с дескрипторами, вариантов получается слишком мало -- мало кто из пользователей будет писать полное название продукта, скорее ограничится только сущностью или только дескриптором. Здесь как раз может помочь бонусное задание -- создать словарь, в котором каждому продукту/бренду будет сопоставлены его возможные наименования: просто сущности, дескрипторы, сущности+дескрипторы -- и оценивать коллокации со всеми возможными упоминаниями товара. Я оценю лучшую метрику после того, как сделаю бонусное. И тогда же посмотрю триграммы.

In [28]:
nes_desc_prep.sort(key=lambda s: len(s.split()), reverse=True)
nes_prep.sort(key=lambda s: len(s.split()), reverse=True)

Даже внутри нашего маленького списка лучших сущности повторяются (la roche-posay, roche-posay). Сначала разберемся с этим. Создаем словарь, где ключом будет самое краткое, но при этом полное название бренда. Значениями будут все возможные варианты названий, включая ключ.

In [30]:
list_remove = []
dict_variants = {}

In [31]:
for item_1 in nes_prep:
  for item_2 in nes_prep:
    if item_1 in item_2 and item_1 != item_2:
      dict_variants[item_2] = [item_1]
      list_remove.append(item_1)
rem_keys = set(nes_prep) - set(list_remove)
print(dict_variants)

{'jane iredale bb': ['jane iredale'], 'escali badger hair': ['badger hair'], 'la roche-posay': ['roche-posay']}


In [32]:
for item_1 in rem_keys:
  for item_2 in rem_keys:
    if item_1 in item_2:
      if item_1 in dict_variants:
        dict_variants[item_1].append(item_2)
      else:
        dict_variants[item_1] = [item_2]
print(dict_variants)

{'jane iredale bb': ['jane iredale', 'jane iredale bb'], 'escali badger hair': ['badger hair', 'escali badger hair'], 'la roche-posay': ['roche-posay', 'la roche-posay'], 'elizabeth arden': ['elizabeth arden'], 'anthony logistics': ['anthony logistics'], 'molton brown': ['molton brown'], 'hot tools': ['hot tools'], 'mario badescu': ['mario badescu'], 'mason pearson': ['mason pearson'], 'juice beauty': ['juice beauty'], 'billy jealousy': ['billy jealousy'], 'bare minerals': ['bare minerals'], 'jack black': ['jack black'], 'paul mitchell': ['paul mitchell']}


Теперь ищем варианты названий по нашим полным спискам сущностей+дескрипторов.

In [33]:
nes_desc_proc_all = [preprocess_text(x) for x in nes_desc]
nes_proc_all = [preprocess_text(x) for x in nes]

In [34]:
for key in dict_variants.keys():
  for ne in nes_desc_proc_all:
    if key in ne:
      dict_variants[key].append(ne)
      dict_variants[key].append(' '.join(ne.replace(key, '').split()))
  for ne in nes_proc_all:
    if key in ne:
      dict_variants[key].append(ne)
      dict_variants[key].append(' '.join(ne.replace(key, '').split()))

Вероятно будут повторы, сразу от них избавимся.

In [35]:
for key in tqdm(dict_variants.keys()):
  dict_variants[key] = set(dict_variants[key])

100%|██████████| 14/14 [00:00<00:00, 18151.55it/s]


Смотрим, что вышло.

In [36]:
dict_variants

{'anthony logistics': {'',
  'anthony logistics',
  'anthony logistics products',
  'products'},
 'bare minerals': {'',
  'bare minerals',
  'bare minerals brushes',
  'bare minerals foundation',
  'bare minerals powder',
  'bare minerals powder foundation',
  'bare minerals powders',
  'bare minerals product',
  'bare minerals products',
  'bare minerals ready',
  'bare minerals ready foundation',
  'brushes',
  'foundation',
  'powder',
  'powder foundation',
  'powders',
  'product',
  'products',
  'ready',
  'ready foundation'},
 'billy jealousy': {'',
  'billy jealousy',
  'billy jealousy moisturizer',
  'billy jealousy product',
  'billy jealousy products',
  'moisturizer',
  'product',
  'products'},
 'elizabeth arden': {'',
  'elizabeth arden',
  'elizabeth arden mascara',
  'elizabeth arden perfume',
  'elizabeth arden product',
  'elizabeth arden products',
  'elizabeth arden sunflowers',
  'elizabeth arden sunflowers perfume',
  'mascara',
  'perfume',
  'product',
  'produ

In [37]:
named_products = []
no_name_products = []
dict_variants['jack black'] = [x for x in dict_variants['jack black'] if len(x) > 2]
for value in dict_variants['jack black']:
  if 'jack black' in value:
    named_products.append(value)
  else:
    no_name_products.append(value)

In [38]:
named_products.sort(key=lambda s: len(s.split()), reverse=True)

In [39]:
no_name_products.sort(key=lambda s: len(s.split()), reverse=True)

In [40]:
named_products

['jack black beard lube conditioning cream',
 'jack black lube conditioning shave product',
 'jack black true volume thickening shampoo',
 'jack black true volume thickening',
 'jack black beard lube conditioning',
 'jack black lube conditioning shave',
 'jack black true volume shampoo',
 'jack black epic moisture lotion',
 'jack black face lotion',
 'jack black epic moisture',
 'jack black lip balms',
 'jack black true volume',
 'jack black daily moisturizer',
 'jack black body wash',
 'jack black lip balm',
 'jack black wax',
 'jack black cream',
 'jack black daily',
 'jack black shampoo',
 'jack black product',
 'jack blacks products',
 'jack black products',
 'jack blacks',
 'jack black']

In [41]:
no_name_products

['beard lube conditioning cream',
 'lube conditioning shave product',
 'true volume thickening shampoo',
 'lube conditioning shave',
 'true volume shampoo',
 'beard lube conditioning',
 'true volume thickening',
 'epic moisture lotion',
 'daily moisturizer',
 'lip balm',
 'epic moisture',
 'true volume',
 's products',
 'face lotion',
 'lip balms',
 'body wash',
 'daily',
 'cream',
 'wax',
 'products',
 'product',
 'shampoo']

Покажу, как мы можем использовать эту группировку по названию бренда в случае если нам нжуно найти все нграммы с упоминанием бренда. Заменяем все возможные названия продуктов с именем бренда на флажок **jack_black_brand** , затем названия без упоминания (из второго списка) заменяем только в том случае, если они встречаются в одном комментарии с названиями из первого списка.

In [148]:
jack_sents = []
for chunk in tqdm(chunks_prep1):
  sents = chunk.split('///')
  for sent in sents:
    for product in named_products:
      if product in sent:
        jack_sents.append(sent.replace(product, 'jack_black_brand'))
jack_sents = list(set(jack_sents))


100%|██████████| 558/558 [00:00<00:00, 1623.24it/s]


In [119]:
jack_sents_new = []
for sent in jack_sents:
  for product in no_name_products:
    if product in sent:
      jack_sents_new.append(sent.replace(product, 'jack_black_brand'))
    else:
      jack_sents_new.append(sent)
jack_sents_new = list(set(jack_sents_new))

In [176]:
list_metrics_jack= {}

bigram_measures = nltk.collocations.BigramAssocMeasures()
creature_filter = lambda *w: 'jack_black_brand' not in w 
finder = BigramCollocationFinder.from_words(' '.join(jack_sents_new).split())
finder.apply_freq_filter(5)
finder.apply_ngram_filter(creature_filter)
list_metrics_jack['jack_black'] = (finder.nbest(bigram_measures.pmi, 15), finder.nbest(bigram_measures.likelihood_ratio, 15), finder.nbest(bigram_measures.jaccard, 15))

In [177]:
print('pmi: ')
for gram in list_metrics_jack['jack_black'][0]:
  print(gram)
print('\nlikelihood_ratio: ')
for gram in list_metrics_jack['jack_black'][1]:
  print(gram)
print('\njaccard: ')
for gram in list_metrics_jack['jack_black'][2]:
  print(gram)

pmi: 
('jack_black_brand', 'sleek')
('conjunction', 'jack_black_brand')
('jack_black_brand', 'industrial')
('">', 'jack_black_brand')
('stick', 'jack_black_brand')
('jack_black_brand', 'beard')
('experience', 'jack_black_brand')
('fan', 'jack_black_brand')
('jack_black_brand', 'turned')
('jack_black_brand', 'easier')
('jack_black_brand', 'clay')
('jack_black_brand', 'customer')
('love', 'jack_black_brand')
('impressive', 'jack_black_brand')
('jack_black_brand', 'true')

likelihood_ratio: 
('love', 'jack_black_brand')
('">', 'jack_black_brand')
('using', 'jack_black_brand')
('conjunction', 'jack_black_brand')
('jack_black_brand', 'beard')
('jack_black_brand', 'true')
('jack_black_brand', 'products')
('fan', 'jack_black_brand')
('jack_black_brand', 'thickening')
('jack_black_brand', 'line')
('jack_black_brand', 'jack_black_brand')
('jack_black_brand', 'shampoo')
('like', 'jack_black_brand')
('jack_black_brand', 'turned')
('jack_black_brand', 'jack_black_brands')

jaccard: 
('using', 'jac

In [178]:
list_metrics_jack= {}

bigram_measures = nltk.collocations.TrigramAssocMeasures()
creature_filter = lambda *w: 'jack_black_brand' not in w 
finder = TrigramCollocationFinder.from_words(' '.join(jack_sents_new).split())
finder.apply_freq_filter(5)
finder.apply_ngram_filter(creature_filter)
list_metrics_jack['jack_black_tri'] = (finder.nbest(bigram_measures.pmi, 15), finder.nbest(bigram_measures.likelihood_ratio, 15), finder.nbest(bigram_measures.jaccard, 15))

In [179]:
print('pmi: ')
for gram in list_metrics_jack['jack_black_tri'][0]:
  print(gram)
print('\nlikelihood_ratio: ')
for gram in list_metrics_jack['jack_black_tri'][1]:
  print(gram)
print('\njaccard: ')
for gram in list_metrics_jack['jack_black_tri'][2]:
  print(gram)

pmi: 
('jack_black_brand', 'industrial', 'strength')
('jack_black_brand', 'sleek', 'finish')
('pairs', 'nicely', 'jack_black_brand')
('jack_black_brand', 'past', 'compare')
("i'll", 'stick', 'jack_black_brand')
('jack_black_brand', 'shea', 'butter')
('originally', 'bought', 'jack_black_brand')
('gave', 'sample', 'jack_black_brand')
('utf8', '">', 'jack_black_brand')
('jack_black_brand', 'beard', 'lube')
('big', 'fan', 'jack_black_brand')
('jack_black_brand', 'intense', 'therapy')
('designer', 'wish', 'jack_black_brand')
('often', 'purchase', 'jack_black_brand')
('think', 'jack_black_brand', 'easier')

likelihood_ratio: 
('jack', 'black', 'jack_black_brand')
('utf8', '">', 'jack_black_brand')
('jack_black_brand', 'true', 'volume')
('jack_black_brand', 'lip', 'balm')
('jack_black_brand', 'beard', 'lube')
("i've", 'using', 'jack_black_brand')
('jack_black_brand', 'expansion', 'technology')
('jack_black_brand', 'thickening', 'shampoo')
('volume', 'thickening', 'jack_black_brand')
('jack_bl

Поскольку теперь у нас очень много результатов и следовтаельно нграмм, мы повысили значение частотного фильтра до 5. Судя по  результатам, лучше всего нам подойдет метрика PMI -- в jaccard мы видим огромное количество мусора и странных вхождений типа ('jack_black_brand', 'jack_black_brand'), он переоценивает всякий мусор типа запятых -- от мусора, конечно, можно было бы почистить корпус перед поиском биграмм, но все же странно оценивать его так высоко. То же в меньшей степени касается и log_likelihood ratio. PMI при это странные символы и знаки препинания не помешали выдать верные информативные нграммы: ('big', 'fan', 'jack_black_brand'), ("i'll", 'stick', 'jack_black_brand'), ('often', 'purchase', 'jack_black_brand').